# Method 1: Snowflake Cortex ML Forecasting
ThisIsClay Co - HVAC Demand Forecasting

This script demonstrates SQL-based forecasting using Snowflake's Cortex ML functions.

What is Cortex ML?
- SQL-native: No Python or coding required (but we'll use Python to execute SQL)
- Fully managed: Snowflake handles model training and deployment
- Quick setup: Simple SQL function calls
- Best for: Analysts who prefer SQL, rapid prototyping

Steps:
1. Explore the HVAC demand data
2. Prepare data for Cortex ML
3. Train forecasting models using SQL
4. Generate 1-year (52 week) forecasts
5. Analyze results by region, product, and customer segment

In [ ]:
import snowflake.snowpark as snowpark
from snowflake.snowpark import Session
import pandas as pd
# import matplotlib.pyplot as plt  # Not available in Snowflake by default
# import seaborn as sns  # Not available in Snowflake by default
from datetime import datetime, timedelta

# Set visualization style
# sns.set_style('whitegrid')  # Not available in Snowflake by default
# plt.rcParams['figure.figsize'] = (14, 6)  # Not available in Snowflake by default

def main(session: Session):
    """
    Main function for Cortex ML forecasting
    """
    
    print("="*80)
    print("METHOD 1: SNOWFLAKE CORTEX ML FORECASTING")
    print("="*80)
    
    # Set context
    session.sql("USE ROLE HVAC_FORECAST_ROLE").collect()
    session.sql("USE WAREHOUSE HVAC_FORECAST_WH").collect()
    session.sql("USE DATABASE HVAC_FORECAST_DB").collect()
    session.sql("USE SCHEMA FORECAST_DATA").collect()
    
    print("\n✓ Connected to Snowflake")
    print("Database: HVAC_FORECAST_DB | Schema: FORECAST_DATA")
    
    # ====================================================================================
    # STEP 1: EXPLORE THE DATA
    # ====================================================================================
    
    print("\n" + "="*80)
    print("STEP 1: DATA EXPLORATION")
    print("="*80)
    
    # Get data summary
    summary_query = """
    SELECT 
        COUNT(*) AS TOTAL_RECORDS,
        COUNT(DISTINCT WEEK_START_DATE) AS TOTAL_WEEKS,
        MIN(WEEK_START_DATE) AS START_DATE,
        MAX(WEEK_START_DATE) AS END_DATE,
        COUNT(DISTINCT REGION) AS NUM_REGIONS,
        COUNT(DISTINCT PRODUCT) AS NUM_PRODUCTS,
        COUNT(DISTINCT CUSTOMER_SEGMENT) AS NUM_SEGMENTS,
        SUM(DEMAND_UNITS) AS TOTAL_DEMAND,
        ROUND(SUM(REVENUE), 2) AS TOTAL_REVENUE
    FROM HVAC_DEMAND_RAW
    """
    
    df_summary = session.sql(summary_query).to_pandas()
    print("\nDataset Summary:")
    for col in df_summary.columns:
        print(f"  {col}: {df_summary[col].values[0]}")
    
    # Customer segment analysis
    segment_query = """
    SELECT 
        CUSTOMER_SEGMENT,
        SUM(DEMAND_UNITS) AS TOTAL_DEMAND,
        ROUND(SUM(REVENUE), 2) AS TOTAL_REVENUE
    FROM HVAC_DEMAND_RAW
    GROUP BY CUSTOMER_SEGMENT
    ORDER BY TOTAL_REVENUE DESC
    """
    
    df_segments = session.sql(segment_query).to_pandas()
    print("\nDemand by Customer Segment:")
    print(df_segments.to_string(index=False))
    
    # Regional analysis
    region_query = """
    SELECT 
        REGION,
        SUM(DEMAND_UNITS) AS TOTAL_DEMAND,
        ROUND(SUM(REVENUE), 2) AS TOTAL_REVENUE
    FROM HVAC_DEMAND_RAW
    GROUP BY REGION
    ORDER BY TOTAL_REVENUE DESC
    """
    
    df_regions = session.sql(region_query).to_pandas()
    print("\nDemand by Region (Top 5):")
    print(df_regions.head().to_string(index=False))
    
    # ====================================================================================
    # STEP 2: PREPARE DATA FOR CORTEX ML
    # ====================================================================================
    
    print("\n" + "="*80)
    print("STEP 2: DATA PREPARATION")
    print("="*80)
    
    # Create input table aggregated by Region, Product, and Customer Segment
    # This gives us detailed forecasts for each combination
    create_input_table = """
    CREATE OR REPLACE TABLE CORTEX_ML_INPUT AS
    SELECT 
        WEEK_START_DATE AS DS,
        REGION,
        PRODUCT,
        CUSTOMER_SEGMENT,
        DEMAND_UNITS AS Y,
        AVG_TEMPERATURE_F,
        ECONOMIC_INDEX,
        IS_WINTER,
        IS_SPRING,
        IS_SUMMER,
        IS_FALL
    FROM HVAC_DEMAND_RAW
    WHERE DEMAND_UNITS > 0  -- Remove zero demand records for cleaner series
    ORDER BY DS, REGION, PRODUCT, CUSTOMER_SEGMENT
    """
    
    session.sql(create_input_table).collect()
    print("✓ Created CORTEX_ML_INPUT table")
    
    # Create series identifier (combination of region, product, segment)
    create_series_table = """
    CREATE OR REPLACE TABLE CORTEX_ML_SERIES AS
    SELECT 
        DS,
        CONCAT(REGION, '_', PRODUCT, '_', CUSTOMER_SEGMENT) AS SERIES_ID,
        REGION,
        PRODUCT,
        CUSTOMER_SEGMENT,
        Y
    FROM CORTEX_ML_INPUT
    ORDER BY DS, SERIES_ID
    """
    
    session.sql(create_series_table).collect()
    print("✓ Created CORTEX_ML_SERIES table with series identifiers")
    
    # Count unique series
    series_count = session.sql("""
        SELECT COUNT(DISTINCT SERIES_ID) AS NUM_SERIES
        FROM CORTEX_ML_SERIES
    """).to_pandas()
    
    print(f"✓ Total time series to forecast: {series_count['NUM_SERIES'].values[0]}")
    
    # Create training view (excluding last 26 weeks for validation)
    train_query = """
    CREATE OR REPLACE VIEW CORTEX_ML_TRAIN AS
    SELECT * FROM CORTEX_ML_SERIES
    WHERE DS <= DATEADD('week', -26, (SELECT MAX(DS) FROM CORTEX_ML_SERIES))
    """
    
    session.sql(train_query).collect()
    
    train_stats = session.sql("""
    SELECT 
        COUNT(DISTINCT DS) AS NUM_WEEKS,
        MIN(DS) AS START_DATE,
        MAX(DS) AS END_DATE,
        COUNT(DISTINCT SERIES_ID) AS NUM_SERIES
    FROM CORTEX_ML_TRAIN
    """).to_pandas()
    
    print(f"\nTraining Data:")
    print(f"  Start: {train_stats['START_DATE'].values[0]}")
    print(f"  End: {train_stats['END_DATE'].values[0]}")
    print(f"  Weeks: {train_stats['NUM_WEEKS'].values[0]}")
    print(f"  Series: {train_stats['NUM_SERIES'].values[0]}")
    
    # ====================================================================================
    # STEP 3: TRAIN CORTEX ML MODEL
    # ====================================================================================
    
    print("\n" + "="*80)
    print("STEP 3: TRAIN CORTEX ML FORECASTING MODEL")
    print("="*80)
    print("\nNOTE: This step uses Snowflake Cortex ML functions.")
    print("Training may take 3-5 minutes...\n")
    
    # Note: Cortex ML forecasting is SQL-based
    # The actual SQL syntax depends on Snowflake version and Cortex availability
    # This is a template that shows the general approach
    
    forecast_sql_template = """
    -- Example Cortex ML Forecast SQL (syntax may vary by Snowflake version)
    -- 
    -- CREATE SNOWFLAKE.ML.FORECAST model_name (
    --     INPUT_DATA => SYSTEM$REFERENCE('VIEW', 'CORTEX_ML_TRAIN'),
    --     SERIES_COLNAME => 'SERIES_ID',
    --     TIMESTAMP_COLNAME => 'DS',
    --     TARGET_COLNAME => 'Y'
    -- );
    --
    -- Then call: CALL model_name!FORECAST(FORECASTING_PERIODS => 52);
    """
    
    print("Cortex ML Forecasting SQL Template:")
    print(forecast_sql_template)
    
    print("\n⚠️  IMPORTANT: Cortex ML functions may not be available in all Snowflake accounts.")
    print("Please check with your Snowflake administrator about ML function availability.")
    print("If available, uncomment and modify the SQL above to train the model.\n")
    
    # ====================================================================================
    # STEP 4: ALTERNATIVE - USE STATISTICAL FORECASTING
    # ====================================================================================
    
    print("="*80)
    print("STEP 4: GENERATE FORECASTS USING SIMPLE STATISTICAL METHOD")
    print("(As alternative to Cortex ML)")
    print("="*80)
    
    # Simple forecasting approach: use last year average with seasonal adjustment
    # This demonstrates the concept even if Cortex ML is not available
    
    forecast_alternative = """
    CREATE OR REPLACE TABLE CORTEX_ML_FORECASTS AS
    WITH historical_avg AS (
        SELECT 
            SERIES_ID,
            REGION,
            PRODUCT,
            CUSTOMER_SEGMENT,
            WEEKOFYEAR,
            AVG(Y) AS AVG_DEMAND,
            STDDEV(Y) AS STDDEV_DEMAND
        FROM (
            SELECT 
                CONCAT(REGION, '_', PRODUCT, '_', CUSTOMER_SEGMENT) AS SERIES_ID,
                REGION,
                PRODUCT,
                CUSTOMER_SEGMENT,
                WEEKOFYEAR(DS) AS WEEKOFYEAR,
                Y
            FROM CORTEX_ML_INPUT
            WHERE DS <= DATEADD('week', -26, (SELECT MAX(DS) FROM CORTEX_ML_INPUT))
        )
        GROUP BY SERIES_ID, REGION, PRODUCT, CUSTOMER_SEGMENT, WEEKOFYEAR
    ),
    forecast_dates AS (
        SELECT 
            DATEADD('week', ROW_NUMBER() OVER (ORDER BY SEQ4()) - 1, 
                    (SELECT DATEADD('week', 1, MAX(DS)) FROM CORTEX_ML_INPUT)) AS FORECAST_DATE
        FROM TABLE(GENERATOR(ROWCOUNT => 52))
    )
    SELECT 
        CURRENT_TIMESTAMP() AS FORECAST_RUN_DATE,
        fd.FORECAST_DATE AS WEEK_START_DATE,
        ha.REGION,
        ha.PRODUCT,
        ha.CUSTOMER_SEGMENT,
        ROUND(ha.AVG_DEMAND * 1.10, 2) AS FORECAST_DEMAND,  -- 10% growth assumption
        ROUND(ha.AVG_DEMAND * 1.10 - ha.STDDEV_DEMAND, 2) AS LOWER_BOUND,
        ROUND(ha.AVG_DEMAND * 1.10 + ha.STDDEV_DEMAND, 2) AS UPPER_BOUND,
        'STATISTICAL_BASELINE' AS METHOD
    FROM forecast_dates fd
    CROSS JOIN historical_avg ha
    WHERE WEEKOFYEAR(fd.FORECAST_DATE) = ha.WEEKOFYEAR
    ORDER BY fd.FORECAST_DATE, ha.REGION, ha.PRODUCT, ha.CUSTOMER_SEGMENT
    """
    
    session.sql(forecast_alternative).collect()
    print("\n✓ Generated 52-week forecasts using statistical baseline method")
    
    # Get forecast summary
    forecast_summary = session.sql("""
    SELECT 
        COUNT(*) AS TOTAL_FORECASTS,
        COUNT(DISTINCT CONCAT(REGION, PRODUCT, CUSTOMER_SEGMENT)) AS NUM_SERIES,
        MIN(WEEK_START_DATE) AS FORECAST_START,
        MAX(WEEK_START_DATE) AS FORECAST_END,
        ROUND(SUM(FORECAST_DEMAND), 0) AS TOTAL_FORECAST_DEMAND
    FROM CORTEX_ML_FORECASTS
    """).to_pandas()
    
    print("\nForecast Summary:")
    for col in forecast_summary.columns:
        print(f"  {col}: {forecast_summary[col].values[0]}")
    
    # ====================================================================================
    # STEP 5: ANALYZE FORECAST RESULTS
    # ====================================================================================
    
    print("\n" + "="*80)
    print("STEP 5: FORECAST ANALYSIS")
    print("="*80)
    
    # Regional forecast totals
    regional_forecast = """
    SELECT 
        REGION,
        ROUND(SUM(FORECAST_DEMAND), 0) AS TOTAL_FORECAST_DEMAND,
        ROUND(AVG(FORECAST_DEMAND), 0) AS AVG_WEEKLY_DEMAND
    FROM CORTEX_ML_FORECASTS
    GROUP BY REGION
    ORDER BY TOTAL_FORECAST_DEMAND DESC
    """
    
    df_regional = session.sql(regional_forecast).to_pandas()
    print("\nForecasted Demand by Region (Next 52 Weeks):")
    print(df_regional.to_string(index=False))
    
    # Product forecast totals
    product_forecast = """
    SELECT 
        PRODUCT,
        ROUND(SUM(FORECAST_DEMAND), 0) AS TOTAL_FORECAST_DEMAND
    FROM CORTEX_ML_FORECASTS
    GROUP BY PRODUCT
    ORDER BY TOTAL_FORECAST_DEMAND DESC
    """
    
    df_product = session.sql(product_forecast).to_pandas()
    print("\nForecasted Demand by Product (Next 52 Weeks):")
    print(df_product.to_string(index=False))
    
    # Customer segment forecasts
    segment_forecast = """
    SELECT 
        CUSTOMER_SEGMENT,
        ROUND(SUM(FORECAST_DEMAND), 0) AS TOTAL_FORECAST_DEMAND
    FROM CORTEX_ML_FORECASTS
    GROUP BY CUSTOMER_SEGMENT
    ORDER BY TOTAL_FORECAST_DEMAND DESC
    """
    
    df_segment = session.sql(segment_forecast).to_pandas()
    print("\nForecasted Demand by Customer Segment (Next 52 Weeks):")
    print(df_segment.to_string(index=False))
    
    # ====================================================================================
    # STEP 6: KEY INSIGHTS
    # ====================================================================================
    
    print("\n" + "="*80)
    print("📊 KEY INSIGHTS - CORTEX ML FORECAST")
    print("="*80)
    
    total_forecast = df_regional['TOTAL_FORECAST_DEMAND'].sum()
    top_region = df_regional.iloc[0]['REGION']
    top_region_demand = df_regional.iloc[0]['TOTAL_FORECAST_DEMAND']
    top_product = df_product.iloc[0]['PRODUCT']
    top_product_demand = df_product.iloc[0]['TOTAL_FORECAST_DEMAND']
    
    print(f"\n1. Total forecasted demand for next year: {total_forecast:,.0f} units")
    print(f"2. Top region: {top_region} ({top_region_demand:,.0f} units)")
    print(f"3. Top product: {top_product} ({top_product_demand:,.0f} units)")
    print(f"4. Top customer segment: {df_segment.iloc[0]['CUSTOMER_SEGMENT']} ({df_segment.iloc[0]['TOTAL_FORECAST_DEMAND']:,.0f} units)")
    
    print("\n" + "="*80)
    print("✅ CORTEX ML FORECASTING COMPLETE!")
    print("="*80)
    
    print("\n📌 SUMMARY: Cortex ML Approach")
    print("-" * 80)
    print("\n✅ Pros:")
    print("  • Simple: SQL-only, minimal coding required")
    print("  • Fast: Quick setup and training")
    print("  • Managed: Snowflake handles infrastructure")
    print("  • Great for: Analysts, rapid prototyping, standard forecasting")
    
    print("\n⚠️ Cons:")
    print("  • Limited customization: Can't tune algorithms deeply")
    print("  • Black box: Less control over model internals")
    print("  • Feature constraints: Limited feature engineering options")
    
    print("\n🎯 Best Use Cases:")
    print("  • Quick forecasts for business reporting")
    print("  • Standard time series patterns")
    print("  • Teams primarily using SQL")
    print("  • When speed matters more than customization")
    
    print("\n" + "="*80)
    print("Next: Try Method 2 (XGBoost) for more control!")
    print("="*80 + "\n")
    
    # ====================================================================================
    # VISUAL VALIDATION: CREATE VIEWS FOR CHARTING
    # ====================================================================================
    
    print("\n" + "="*80)
    print("📊 CREATING VISUALIZATION VIEWS")
    print("="*80)
    
    # Create a view for time series visualization
    viz_view = """
    CREATE OR REPLACE VIEW CORTEX_ML_VIZ_TIMESERIES AS
    SELECT 
        WEEK_START_DATE,
        SUM(FORECAST_DEMAND) AS TOTAL_WEEKLY_FORECAST,
        AVG(FORECAST_DEMAND) AS AVG_FORECAST_PER_SERIES
    FROM CORTEX_ML_FORECASTS
    GROUP BY WEEK_START_DATE
    ORDER BY WEEK_START_DATE
    """
    session.sql(viz_view).collect()
    
    # Create a view for regional comparison
    viz_regional = """
    CREATE OR REPLACE VIEW CORTEX_ML_VIZ_REGIONAL AS
    SELECT 
        REGION,
        SUM(FORECAST_DEMAND) AS TOTAL_FORECAST,
        COUNT(DISTINCT PRODUCT) AS NUM_PRODUCTS,
        COUNT(DISTINCT CUSTOMER_SEGMENT) AS NUM_SEGMENTS
    FROM CORTEX_ML_FORECASTS
    GROUP BY REGION
    ORDER BY TOTAL_FORECAST DESC
    """
    session.sql(viz_regional).collect()
    
    print("\n✅ Created visualization views!")
    print("\nYou can now create charts in Snowsight using:")
    print("  • CORTEX_ML_VIZ_TIMESERIES - Weekly forecast trend")
    print("  • CORTEX_ML_VIZ_REGIONAL - Regional comparison")
    
    # Display sample validation data
    print("\n" + "="*80)
    print("📈 VALIDATION: SAMPLE FORECAST DATA")
    print("="*80)
    
    sample_data = session.sql("""
        SELECT 
            WEEK_START_DATE,
            REGION,
            PRODUCT,
            CUSTOMER_SEGMENT,
            FORECAST_DEMAND,
            METHOD
        FROM CORTEX_ML_FORECASTS
        WHERE WEEK_START_DATE <= (SELECT MIN(WEEK_START_DATE) + INTERVAL '3 weeks' FROM CORTEX_ML_FORECASTS)
        ORDER BY WEEK_START_DATE, REGION, PRODUCT
        LIMIT 10
    """).to_pandas()
    
    print("\nSample Forecasts (First 3 Weeks):")
    print(sample_data.to_string(index=False))
    
    # Validation checks
    print("\n" + "="*80)
    print("✅ VALIDATION CHECKS")
    print("="*80)
    
    checks = session.sql("""
        SELECT 
            COUNT(*) AS TOTAL_FORECASTS,
            COUNT(DISTINCT WEEK_START_DATE) AS UNIQUE_WEEKS,
            COUNT(DISTINCT REGION) AS UNIQUE_REGIONS,
            COUNT(DISTINCT PRODUCT) AS UNIQUE_PRODUCTS,
            MIN(FORECAST_DEMAND) AS MIN_FORECAST,
            MAX(FORECAST_DEMAND) AS MAX_FORECAST,
            AVG(FORECAST_DEMAND) AS AVG_FORECAST,
            CASE 
                WHEN COUNT(*) >= 52 THEN '✅ PASS'
                ELSE '❌ FAIL'
            END AS WEEKS_CHECK,
            CASE 
                WHEN MIN(FORECAST_DEMAND) >= 0 THEN '✅ PASS'
                ELSE '❌ FAIL'
            END AS POSITIVE_CHECK
        FROM CORTEX_ML_FORECASTS
    """).to_pandas()
    
    print("\n🔍 Data Quality Checks:")
    for col in checks.columns:
        val = checks[col].values[0]
        if col.endswith('_CHECK'):
            print(f"  {col}: {val}")
        else:
            print(f"  {col}: {val}")
    
    print("\n" + "="*80)
    print("🎯 TO VISUALIZE IN SNOWSIGHT:")
    print("="*80)
    print("""
1. Go to Worksheets in Snowsight
2. Run: SELECT * FROM CORTEX_ML_VIZ_TIMESERIES
3. Click 'Chart' button
4. Select 'Line Chart'
5. X-axis: WEEK_START_DATE
6. Y-axis: TOTAL_WEEKLY_FORECAST
    
This will show your 52-week forecast trend! 📈
    """)
    
    return session

# For Snowflake Notebooks

In [ ]:
# Get active session and run main function
session = snowpark.context.get_active_session()
main(session)

## Test with SQL

In [ ]:
SELECT * FROM CORTEX_ML_VIZ_TIMESERIES;